# Weather Data Exploration

This notebook performs analysis on the first 2016 rows of the weather.csv file, skipping the 'date' and 'OT' columns.

## Analyses:
1. Time-Series Line Charts (One Chart per Variable)
2. Rolling Average (24-Sample i.e. 4 hrs Window)
3. Histogram (Distribution)
4. Box Plot (Variability)
5. Scatter Plot (Correlation: Temperature vs RH and other correlated variables)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.stattools import acf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load data
df = pd.read_csv('/content/PatchTST/datasets/weather/weather.csv')
df = df.iloc[:2016].drop(['date', 'OT'], axis=1)
print(f"Data shape: {df.shape}")
print("Columns:", list(df.columns))
df.head()

## 1. Time-Series Line Charts (One Chart per Variable)

In [ ]:
# Time-Series Line Charts in grids of 4
n_cols = 4
n_rows = (len(df.columns) + n_cols - 1) // n_cols  # Ceiling division

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
fig.suptitle('Time Series of All Variables', fontsize=16, y=0.98)

for i, col in enumerate(df.columns):
    row = i // n_cols
    col_pos = i % n_cols
    
    if n_rows == 1:
        ax = axes[col_pos]
    else:
        ax = axes[row, col_pos]
    
    ax.plot(df.index, df[col], linewidth=1.5)
    ax.set_title(f'{col}', fontsize=10)
    ax.set_xlabel('Time Index', fontsize=8)
    ax.set_ylabel(col, fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='both', which='major', labelsize=8)

# Hide empty subplots
total_plots = n_rows * n_cols
for i in range(len(df.columns), total_plots):
    row = i // n_cols
    col_pos = i % n_cols
    if n_rows == 1:
        axes[col_pos].set_visible(False)
    else:
        axes[row, col_pos].set_visible(False)

plt.tight_layout()
plt.show()

## 2. Rolling Average (24-Sample Window)

In [ ]:
# Rolling Average Charts in grids of 4
n_cols = 4
n_rows = (len(df.columns) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
fig.suptitle('Time Series with Rolling Average (24-Sample Window)', fontsize=16, y=0.98)

for i, col in enumerate(df.columns):
    row = i // n_cols
    col_pos = i % n_cols
    
    if n_rows == 1:
        ax = axes[col_pos]
    else:
        ax = axes[row, col_pos]
    
    ax.plot(df.index, df[col], label=col, alpha=0.7, linewidth=1)
    ax.plot(df.index, df[col].rolling(window=24).mean(), label='Rolling Avg (24)', color='red', linewidth=2)
    ax.set_title(f'{col}', fontsize=10)
    ax.set_xlabel('Time Index', fontsize=8)
    ax.set_ylabel(col, fontsize=8)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='both', which='major', labelsize=8)

# Hide empty subplots
total_plots = n_rows * n_cols
for i in range(len(df.columns), total_plots):
    row = i // n_cols
    col_pos = i % n_cols
    if n_rows == 1:
        axes[col_pos].set_visible(False)
    else:
        axes[row, col_pos].set_visible(False)

plt.tight_layout()
plt.show()

## 3. Histograms (Distribution)

In [ ]:
df.hist(figsize=(16, 12), bins=30)
plt.suptitle('Histograms of All Variables')
plt.show()

## 4. Box Plots (Variability)

In [ ]:
plt.figure(figsize=(16, 8))
df.boxplot(rot=45)
plt.title('Box Plots of All Variables')
plt.show()

## 5. Heatmap Correlation

In [ ]:
# Correlation matrix
corr = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

# Find highly correlated pairs (|corr| > 0.8)
high_corr = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        if abs(corr.iloc[i, j]) > 0.8:
            high_corr.append((corr.columns[i], corr.columns[j], corr.iloc[i, j]))

print("Highly correlated pairs (|corr| > 0.8):")
for pair in high_corr:
    print(f"{pair[0]} vs {pair[1]}: {pair[2]:.2f}")

## 6. Scatter Plot

In [ ]:
# Scatter plots for highly correlated pairs in a grid
if high_corr:
    n_pairs = len(high_corr)
    n_cols = min(4, n_pairs)  # Max 4 columns
    n_rows = (n_pairs + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
    fig.suptitle('Scatter Plots for Highly Correlated Pairs (|corr| > 0.8)', fontsize=14, y=0.98)
    
    for i, pair in enumerate(high_corr):
        row = i // n_cols
        col_pos = i % n_cols
        
        if n_rows == 1 and n_cols == 1:
            ax = axes
        elif n_rows == 1:
            ax = axes[col_pos]
        else:
            ax = axes[row, col_pos]
        
        sns.scatterplot(x=df[pair[0]], y=df[pair[1]], ax=ax, alpha=0.6)
        ax.set_title(f'{pair[0]} vs {pair[1]}\n(corr: {pair[2]:.2f})', fontsize=10)
        ax.set_xlabel(pair[0], fontsize=8)
        ax.set_ylabel(pair[1], fontsize=8)
        ax.grid(True, alpha=0.3)
    
    # Hide empty subplots
    total_plots = n_rows * n_cols
    for i in range(n_pairs, total_plots):
        row = i // n_cols
        col_pos = i % n_cols
        if n_rows == 1 and n_cols == 1:
            pass  # Only one plot
        elif n_rows == 1:
            axes[col_pos].set_visible(False)
        else:
            axes[row, col_pos].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("No highly correlated pairs found (|corr| > 0.8)")

## 7. Autocorrelation Function (ACF)

In [ ]:
# ACF for key variables in a grid
key_vars = ['T (degC)', 'rh (%)', 'wv (m/s)']
n_vars = len([col for col in key_vars if col in df.columns])

fig, axes = plt.subplots(1, n_vars, figsize=(6*n_vars, 6))
fig.suptitle('Autocorrelation Function (ACF) for Key Variables', fontsize=14, y=0.98)

for i, col in enumerate(key_vars):
    if col in df.columns:
        acf_vals = acf(df[col], nlags=50)
        axes[i].stem(range(len(acf_vals)), acf_vals, linefmt='b-', markerfmt='bo', basefmt='k-')
        axes[i].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
        axes[i].axhline(y=-1.96/np.sqrt(len(df)), color='r', linestyle='--', linewidth=1, label='95% CI')
        axes[i].axhline(y=1.96/np.sqrt(len(df)), color='r', linestyle='--', linewidth=1)
        axes[i].set_title(f'ACF: {col}', fontsize=12)
        axes[i].set_xlabel('Lag', fontsize=10)
        axes[i].set_ylabel('Autocorrelation', fontsize=10)
        axes[i].grid(True, alpha=0.3)
        axes[i].legend(fontsize=8)

plt.tight_layout()
plt.show()

## 8. Outlier Detection (IQR Method)

In [ ]:
# Outlier detection using IQR method
outliers = {}
for col in df.columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]

print("Number of outliers per variable (IQR method):")
for col, count in outliers.items():
    print(f"{col}: {count}")

# Visualize outliers for key variables in a grid
key_outlier_vars = ['T (degC)', 'rh (%)', 'p (mbar)']
available_vars = [col for col in key_outlier_vars if col in df.columns]

fig, axes = plt.subplots(1, len(available_vars), figsize=(6*len(available_vars), 6))
fig.suptitle('Outlier Detection for Key Variables (IQR Method)', fontsize=14, y=0.98)

for i, col in enumerate(available_vars):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    axes[i].plot(df.index, df[col], label=col, alpha=0.7, linewidth=1)
    axes[i].fill_between(df.index, lower_bound, upper_bound, color='green', alpha=0.1, label='Normal Range')
    outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    axes[i].scatter(df[outlier_mask].index, df[outlier_mask][col], color='red', s=30, label='Outliers', zorder=5)
    axes[i].set_title(f'Outliers: {col}', fontsize=12)
    axes[i].set_xlabel('Time Index', fontsize=10)
    axes[i].set_ylabel(col, fontsize=10)
    axes[i].legend(fontsize=8)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Trend Analysis (Linear Regression)

In [ ]:
# Linear trend analysis for key variables in a grid
fig, axes = plt.subplots(1, len(key_vars), figsize=(6*len(key_vars), 6))
fig.suptitle('Trend Analysis (Linear Regression) for Key Variables', fontsize=14, y=0.98)

for i, col in enumerate(key_vars):
    if col in df.columns:
        X = np.arange(len(df)).reshape(-1, 1)
        y = df[col].values
        model = LinearRegression()
        model.fit(X, y)
        trend = model.predict(X)
        
        axes[i].plot(df.index, df[col], label=col, alpha=0.7, linewidth=1)
        axes[i].plot(df.index, trend, label='Linear Trend', color='red', linewidth=2)
        axes[i].set_title(f'Trend: {col}', fontsize=12)
        axes[i].set_xlabel('Time Index', fontsize=10)
        axes[i].set_ylabel(col, fontsize=10)
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)
        
        print(f"{col} - Trend Slope: {model.coef_[0]:.6f}, Intercept: {model.intercept_:.4f}")

plt.tight_layout()
plt.show()

## 10. Wind Rose Plot

In [ ]:
# Wind Rose Plot
if 'wd (deg)' in df.columns and 'wv (m/s)' in df.columns:
    directions = df['wd (deg)'].values
    speeds = df['wv (m/s)'].values
    
    # Bin directions into 16 sectors (22.5 degrees each)
    bins = np.arange(0, 360, 22.5)
    labels = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    dir_bins = np.digitize(directions, bins) - 1
    dir_bins[dir_bins == 16] = 0  # Wrap around
    dir_bins[dir_bins == -1] = 0  # Handle edge case
    
    # Calculate frequency for each direction
    freq = np.zeros(16)
    for i in range(16):
        freq[i] = np.sum(dir_bins == i)
    
    # Create polar plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, polar=True)
    theta = np.linspace(0, 2*np.pi, 16, endpoint=False)
    bars = ax.bar(theta, freq, width=2*np.pi/16, bottom=0.0, align='center', alpha=0.7, edgecolor='black')
    
    # Color bars by frequency
    colors = plt.cm.viridis(freq / freq.max())
    for bar, color in zip(bars, colors):
        bar.set_facecolor(color)
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_xticks(theta)
    ax.set_xticklabels(labels)
    ax.set_title('Wind Rose - Wind Direction Frequency', pad=20)
    plt.show()
    
    print(f"Most frequent wind direction: {labels[np.argmax(freq)]} ({freq.max()} occurrences)")

## 11. Pair Plot (Correlation Matrix Visualization)

In [ ]:
# Pair plot for selected key variables (to avoid overcrowding)
selected_cols = ['T (degC)', 'rh (%)', 'p (mbar)', 'wv (m/s)', 'Tdew (degC)']
available_cols = [col for col in selected_cols if col in df.columns]

if len(available_cols) > 0:
    sample_df = df[available_cols].sample(min(500, len(df)), random_state=42)
    sns.pairplot(sample_df, diag_kind='kde', corner=True, plot_kws={'alpha': 0.6})
    plt.suptitle('Pair Plot of Key Variables (Sampled 500 points)', y=1.02)
    plt.show()
else:
    print("Selected columns not found in dataframe")

## 12. Principal Component Analysis (PCA)

In [ ]:
# PCA - Dimensionality Reduction
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# PCA with 2 components for visualization
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], 
                     c=df['T (degC)'] if 'T (degC)' in df.columns else range(len(df)), 
                     cmap='coolwarm', alpha=0.6, edgecolors='k', linewidth=0.5)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA: First Two Principal Components')
plt.colorbar(scatter, label='Temperature (degC)' if 'T (degC)' in df.columns else 'Index')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Explained Variance Ratio (PC1, PC2): {pca.explained_variance_ratio_}")
print(f"Total Explained Variance: {np.sum(pca.explained_variance_ratio_):.2%}")

# Full PCA to see cumulative variance
pca_full = PCA()
pca_full.fit(df_scaled)
cumsum = np.cumsum(pca_full.explained_variance_ratio_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumsum) + 1), cumsum, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA: Cumulative Explained Variance')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
plt.axhline(y=0.90, color='orange', linestyle='--', label='90% Variance')
plt.legend()
plt.show()

n_components_95 = np.argmax(cumsum >= 0.95) + 1
print(f"\nNumber of components needed to explain 95% variance: {n_components_95}")